In [1]:
%load_ext nb_mypy

Version 1.0.5


In [2]:
from loguru import logger
import sys

from sm_pipelines_oo.config_loader.implementations import MockConfigLoader
from sm_pipelines_oo.pipeline import PipelineFacade, DevPipelineFacade

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/thomas-22/.config/sagemaker/config.yaml


In [3]:
_PROJECT_BUCKET = 'example-sm-pipelines-20231117141913829300000001'
_DATA_PREFIX = 'examples/data/test_input_1.parquet'

_shared_config_dict={
    'project_name': 'test',
    'project_version': '0',
    'region': 'us-east-1',
    'project_bucket_name': 'example-sm-pipelines-20231117141913829300000001',
    'role_name': 'sagemaker_pipelines_role'
}
_fw_processor_config_dict = {
    'step_name': 'preprocessing',
    'step_factory_class': 'FrameworkProcessor',
    'processor_init_config': {
        'framework_version': '0.23-1',
        'estimator_cls_name': 'SKLearn',
        'instance_count': 1,
        'instance_type': 'ml.m5.xlarge',
    },
    'processor_run_config': {
        'code': 'preprocess.py',
        'source_dir': 'worker_code/preprocess',
        'input_files_s3paths': [
            f's3://{_PROJECT_BUCKET}/{_DATA_PREFIX}',
        ],
        'output_files_s3paths': [],
    },
    'shared_config': _shared_config_dict
}
_mock_config_loader = MockConfigLoader(
    shared_config_dict=_shared_config_dict,
    step_configs_dicts=[_fw_processor_config_dict,]
)

In [4]:
from pydantic_settings import BaseSettings
from pydantic import field_validator, ValidationError
from s3path import S3Path

class _RunConfig(BaseSettings):
    # todo: allow athena datasetdefinition instead
    input_files_s3paths: list[str]  # todo: validate it's an s3 path

    @field_validator('input_files_s3paths')
    @classmethod
    def parse_str_to_s3path(cls, path_list: list[str]) -> list[S3Path]:
        return [S3Path(path_str) for path_str in path_list]

good_class = _RunConfig(input_files_s3paths=['abc/key.json'])
good_class.input_files_s3paths

[S3Path('abc/key.json')]

In [5]:
pipeline = DevPipelineFacade(
    env='dev',
    # Use different configs for testing
    config_loader=_mock_config_loader,
)


INFO:botocore.tokens:Loading cached SSO token for ml
INFO:botocore.tokens:SSO Token refresh succeeded
/home/thomas-22/.cache/pypoetry/virtualenvs/sm-pipelines-oo-examples-yainlQgE-py3.10/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(


In [6]:
pipeline.create_and_start_pipeline_from_definition()

INFO:botocore.tokens:Loading cached SSO token for ml
INFO:sagemaker.processing:Uploaded worker_code/preprocess to s3://sagemaker-us-east-1-338755209567/test-v0/code/c539bb7cbc8181b1d0f8b983c763d37f/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-east-1-338755209567/test-v0/code/bc2536a25d34e1ecae5238f42f4207c2/runproc.sh
/home/thomas-22/.cache/pypoetry/virtualenvs/sm-pipelines-oo-examples-yainlQgE-py3.10/lib/python3.10/site-packages/sagemaker/workflow/pipeline_context.py:332: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
2024-02-28 20:06:24.135 | INFO     | sm_pipelines_oo.pipeline:export_pipeline_definition_to_s3:77 - Uploaded pipeline definition to s3://example-sm-pipelines-20231117141913829300000001/pipeline_definitions/test-v0-definition.json

An error occurred (ValidationException) when calling the CreatePipeline operation: Pipeline names must be unique within an AWS acc